In [ ]:
import numpy as np
from scipy.special import erf
import os
import sys
import time
import copy
import logging
import matplotlib.pyplot as plt

# Navigate to the parent directory of the project structure
project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_dir = os.path.join(project_dir, 'src')
log_dir = os.path.join(project_dir, 'log')
fig_dir = os.path.join(project_dir, 'fig')

# Add the src directory to sys.path
sys.path.append(src_dir)



from Dataset.flight_dataset_for_LS import load_flight_dataset_matrices_for_LS, save_flight_dataset_matrices_for_LS
from analysis.commons import data_normalize_by_features, concatenate_B_b


from RP_mechanisms.optim_RP_DP import compute_largest_l2
from LS_mechanisms.alt19_LS import ALT19LS_mech
from LS_mechanisms.optim_LS_DP import LS_fromoptim_RP_mech, OptimalLS_mech
from LS_mechanisms.LS_accuracy_estimator import LS_accuracy_estimator

In [2]:
logfile_path = os.path.join(log_dir, 'LS-flight-default.log')
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(message)s",
    handlers=[
        logging.FileHandler(logfile_path),
        logging.StreamHandler()
    ]
)

def store_array_str(data_array):
    converted_str = np.array_str(data_array)
    converted_str = converted_str.replace(' ',',')
    return converted_str

file_X_path = os.path.join(project_dir, "Dataset", "flight-LR-X.txt")
file_y_path = os.path.join(project_dir, "Dataset", "flight-LR-y.txt")

X = None
y = None
try:
    X, y = load_flight_dataset_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
except:
    save_flight_dataset_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
    X, y = load_flight_dataset_matrices_for_LS()
    
B, b = data_normalize_by_features(X, y)
A = concatenate_B_b(B, b)

In [3]:
def generate_default_configuration():
    kwargs = {
        'database': A,
        'r' : 1200,
        'CI' : 0.95,
        'bootstrap_samples': 100, 
        'l2' : compute_largest_l2(A)
    }
    
    return kwargs

kwargs = generate_default_configuration()

In [4]:
mech = OptimalLS_mech(kwargs)

epsilon = 1
delta = 1/mech.n

mech.find_minimal_sigma(epsilon, delta)

0.8641385229788721

In [5]:
mech = OptimalLS_mech(kwargs)

epsilon = 1
delta = 1/mech.n
num_samples = 100

tic = time.perf_counter()
samples = mech._gen_samples(epsilon, delta, num_samples)
toc = time.perf_counter()
logging.critical(f"cost {toc - tic:0.4f} seconds")

2025-10-28 20:13:23,569 cost 2.6036 seconds


In [6]:
tic = time.perf_counter()
accuracy_estimator = LS_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_square_error(samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")

2025-10-28 20:13:23,617 (bootstrap_result(lb_one_sided=np.float64(0.4647020758734411), lb_two_sided=np.float64(0.4640138937866858), ub_one_sided=np.float64(0.47363654914553976), ub_two_sided=np.float64(0.47429460258762646)), bootstrap_result(lb_one_sided=np.float64(1.8308190644943767), lb_two_sided=np.float64(1.8300463881676652), ub_one_sided=np.float64(1.8605492462936053), ub_two_sided=np.float64(1.862951138639569)))
2025-10-28 20:13:23,618 cost 0.0393 seconds


In [7]:
mech = LS_fromoptim_RP_mech(kwargs)

epsilon = 1
delta = 1/mech.n

mech.find_minimal_sigma(epsilon, delta)

/home/wei402/Desktop/NDIS/src/RP_mechanisms/optim_RP.py:25: RuntimeWarning: overflow encountered in exp
  exp_term = np.exp(epsilon - r / 2 * np.log(1 - leverage)) * (1 - leverage) ** (r / 2.0)
/home/wei402/Desktop/NDIS/src/RP_mechanisms/optim_RP.py:25: RuntimeWarning: invalid value encountered in scalar multiply
  exp_term = np.exp(epsilon - r / 2 * np.log(1 - leverage)) * (1 - leverage) ** (r / 2.0)


np.float64(0.7452001524823914)

In [ ]:
mech = LS_fromoptim_RP_mech(kwargs)

epsilon = 1
delta = 1/mech.n
num_samples = 100

tic = time.perf_counter()
samples = mech._gen_samples(epsilon, delta, num_samples)
toc = time.perf_counter()
logging.critical(f"cost {toc - tic:0.4f} seconds")

In [ ]:
tic = time.perf_counter()
accuracy_estimator = LS_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_square_error(samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")

In [ ]:
mech = ALT19LS_mech(kwargs)

epsilon = 1
delta = 1/mech.n

mech.compute_constant(epsilon, delta)

In [ ]:
mech = ALT19LS_mech(kwargs)

epsilon = 1
delta = 1/mech.n
num_samples = 100

tic = time.perf_counter()
samples = mech._gen_samples(epsilon, delta, num_samples)
toc = time.perf_counter()
logging.critical(f"cost {toc - tic:0.4f} seconds")

In [ ]:
tic = time.perf_counter()
accuracy_estimator = LS_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_square_error(samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")